In [ ]:
%pip install contextily

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely import wkb
import math

directory_path = "/content/drive/MyDrive/gpx"
all_files = os.listdir(directory_path)
parquet_files = [os.path.join(directory_path, f) for f in all_files if f.endswith(".parquet")]

all_tracks_df = pd.DataFrame()
for file_path in parquet_files:
    df = pd.read_parquet(file_path)
    all_tracks_df = pd.concat([all_tracks_df, df], ignore_index=True)

if not all_tracks_df.empty:
    all_tracks_df = all_tracks_df[all_tracks_df['city'].str.lower() != 'unknown']
    all_tracks_df['geometry'] = all_tracks_df['geometry'].apply(wkb.loads)
    gdf = gpd.GeoDataFrame(all_tracks_df, geometry='geometry', crs='EPSG:4326')
    gdf = gdf.to_crs(epsg=3857)
    gdf['length_km'] = gdf.geometry.length / 1000
    gdf = gdf[gdf['length_km'] > 10]

    if not gdf.empty:
        cities = sorted(gdf['city'].unique())
        n = len(cities)
        cols = math.ceil(math.sqrt(n))
        rows = math.ceil(n / cols)
        fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 6*rows))
        axes = axes.flatten()

        for i, city in enumerate(cities):
            city_gdf = gdf[gdf['city'] == city]
            city_gdf.plot(ax=axes[i], linewidth=2, color='red')
            ctx.add_basemap(axes[i], source=ctx.providers.OpenStreetMap.Mapnik)
            axes[i].set_axis_off()
            axes[i].set_title(city, fontsize=14)

        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)

        plt.tight_layout()
        plt.show()
